[Reference](https://blog.dataengineerthings.org/how-to-build-a-local-elt-pipeline-with-duckdb-and-dbt-c8080ee89036)

# Step 1: Extract and Load

In [1]:
!pip install duckdb dbt-duckdb pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.7/442.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.

In [3]:
import duckdb

def load_data():
    # Connect to a persistent DuckDB file
    # If the file doesn't exist, this creates it
    con = duckdb.connect('coffeeshop.duckdb')

    print("🚀 Starting ELT Process...")

    # 1. create a schema for raw data
    con.sql("CREATE SCHEMA IF NOT EXISTS raw;")

    # 2. Load Sales Data (CSV)
    # read_csv_auto is a DuckDB magic function that infers types
    print("... Loading Sales Data")
    con.sql("""
        CREATE OR REPLACE TABLE raw.source_sales AS
        SELECT * FROM read_csv_auto('data/sales_data.csv');
    """)

    # 3. Load Budget Data (CSV/Excel converted to CSV for simplicity)
    print("... Loading Budget Data")
    con.sql("""
        CREATE OR REPLACE TABLE raw.source_budget AS
        SELECT * FROM read_csv_auto('data/budget_data.csv');
    """)

    # Validation check - to see if data is loaded into database successfully
    count = con.sql("SELECT count(*) FROM raw.source_sales").fetchone()
    print(f"✅ Loaded {count[0]} sales records.")

    con.close()

if __name__ == "__main__":
    load_data()

# Step 2: dbt Setup and Configuration


In [4]:
!dbt init coffee_analytics

14:40:50  Running with dbt=1.11.2
14:40:50  Creating dbt configuration folder at /root/.dbt
14:40:50  
Your new dbt project "coffee_analytics" was created!

For more information on how to configure the profiles.yml file,
please consult the dbt documentation here:

  https://docs.getdbt.com/docs/configure-your-profile

One more thing:

Need help? Don't hesitate to reach out to us via GitHub issues or on Slack:

  https://community.getdbt.com/

Happy modeling!

14:40:50  Setting up your profile.
Which database would you like to use?
[1] duckdb

(Don't see the one you want? https://docs.getdbt.com/docs/available-adapters)

Enter a number: 
Which database would you like to use?
[1] duckdb

(Don't see the one you want? https://docs.getdbt.com/docs/available-adapters)

Enter a number: 1
14:40:57  Profile coffee_analytics written to /root/.dbt/profiles.yml using target's sample configuration. Once updated, you'll be able to start developing with dbt.


```
#  Set up profiles.yml
coffee_analytics:
  target: dev
  outputs:
    dev:
      type: duckdb
      # This path must point to the yourdatabase
      path: 'absolute/full/path/to/your/project/coffee.duckdb'
      threads: 1
```

```
# sources.yml
version: 2

sources:
  - name: bean_raw
    # Our schema is raw - can be found in step 1
    schema: raw <-- This tells dbt the actual schema name in the database (DuckDB)
    tables:
       # We have two tables - source_sales and source_budget
      - name: source_sales
      - name: source_budget
```      